# Data flow graph

In [3]:
import tensorflow as tf

a = tf.constant(5, name="input_a")
b = tf.constant(3, name="input_b")
c = tf.multiply(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")
e = tf.add(c, d, name="add_e")

sess = tf.Session()
output = sess.run(e)

#writer = tf.train.SummaryWriter("./my_graph", sess.graph) # old version
writer = tf.summary.FileWriter("./my_graph", sess.graph)
sess.close()
writer.close()

print(output)

23


In [3]:
# -----
import tensorflow as tf

a = tf.constant([5, 3], name="input_a")
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")
d = tf.add(c, b, name="add_d")

print(d)

Tensor("add_d_1:0", shape=(), dtype=int32)


In [4]:
tf.get_default_graph()

# Dict

In [10]:
import tensorflow as tf

a = tf.add(2, 5)
b = tf.multiply(a, 3)
sess = tf.Session()

replace_dict = {a: 15}
sess.run(b, feed_dict = replace_dict)

sess.close()

# Placeholder

In [13]:
import tensorflow as tf
import numpy as np

a = tf.placeholder(tf.int32, shape=[2], name="my_input")
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")
d = tf.add(b, c, name="add_d")

In [17]:
my_var = tf.Variable(1)
my_var_times_two = my_var.assign(my_var * 2)
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
sess.run(my_var_times_two)

2

In [18]:
sess.run(my_var_times_two)

4

In [19]:
sess.run(my_var_times_two)

8

# Name scope

In [20]:
import tensorflow as tf

with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name="A_add")
    b = tf.multiply(a, 3, name="A_mul")
with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = tf.multiply(c, 6, name="B_mul")
e = tf.add(b, d, name="output")

writer=tf.summary.FileWriter("./name_scope_1",graph=tf.get_default_graph())
writer.close()

# Variable

In [10]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    #v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1]))
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer)

g2 = tf.Graph()
with g2.as_default():
    #v = tf.get_variable("v", initializer=tf.ones_initializer(shape=[1]))
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer)
    
with tf.Session(graph=g1) as sess:
    #tf.initialize_all_variables().run()  #old version
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
with tf.Session(graph=g2) as sess:
    #tf.initialize_all_variables().run()  #old version
     tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[ 0.]
[ 1.]


# Device used

In [15]:
import tensorflow as tf

a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
g = tf.Graph()
with g.device('/gpu:0'):
    result = a + b
    print(result)

Tensor("add_3:0", shape=(2,), dtype=float32)


# Batch normalization

In [ ]:
fc_mean, fc_var = tf.nn.moments(Wx_plus_b, axes=[0], )
scale = tf.Variable(tf.ones([out_size]))
shift = tf.Variable(tf.zeros([out_size]))
episilon = 0.01
Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, 
                                     scale, epsilon)
# This is
Wx_plus_b = (Wx_plus_b - fc_mean) / tf.sqrt(fc_var + 0.001)
Wx_plus_b = Wx_plus_b * scale + shift

# Activate function

In [20]:
# Sigmoid
import tensorflow as tf

a = tf.constant([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]])
sess = tf.Session()
print(sess.run(tf.sigmoid(a)))

# ReLu
b = tf.constant([-1.0, 2.0])
with tf.Session() as sess:
    print(sess.run(tf.nn.relu(b)))
    
# Dropout
c = tf.constant([[-1.0, 2.0, 3.0, 4.0]])
with tf.Session() as sess:
    print(sess.run(tf.nn.dropout(c, 0.5, noise_shape = [1, 1])))

[[ 0.7310586   0.88079703]
 [ 0.7310586   0.88079703]
 [ 0.7310586   0.88079703]]
[ 0.  2.]
[[-2.  4.  6.  8.]]


# Convolution

In [23]:
# conv2d
import numpy as np

input_data = tf.Variable(np.random.rand(10, 9, 9, 3), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(2, 2, 3, 2), dtype=np.float32)
y = tf.nn.conv2d(input_data, filter_data, strides=[1,1,1,1], padding="SAME")
print(y)

Tensor("Conv2D_1:0", shape=(10, 9, 9, 2), dtype=float32)


# Load model

In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    ckpt = tf.train().get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)

# Queue

In [24]:
# FIFOQueue
import tensorflow as tf

q = tf.FIFOQueue(3, "float")
init = q.enqueue_many(([0.1, 0.2, 0.3], ))
x = q.dequeue()
y = x + 1
q_inc = q.enqueue([y])

with tf.Session() as sess:
    sess.run(init)
    quelen = sess.run(q.size())
    for i in range(2):
        sess.run(q_inc)
        
    quelen = sess.run(q.size())
    for i in range(quelen):
        print(sess.run(q.dequeue()))

0.3
1.1
1.2


In [26]:
# RandomShuffleQueue
q = tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes="float")
sess = tf.Session()
for i in range(0, 10):
    sess.run(q.enqueue(i))
for i in range(0, 8):
    print(sess.run(q.dequeue()))

5.0
1.0
2.0
9.0
8.0
3.0
6.0
0.0


# Load datesets

In [27]:
# 1.preloaded data
x1 = tf.constant([2, 3, 4])
x2 = tf.constant([4, 0, 1])
y = tf.add(x1, x2)

In [ ]:
# 2.feeding
import tensorflow as tf

a1 = tf.placeholder(tf.int16)
a2 = tf.placeholder(tf.int16)
b = tf.add(x1, x2)

li1 = [2, 3, 4]
li2 = [4, 0, 1]
with tf.Session() as sess:
    print(sess.run(b, feed_dict={a1:li1, a2:li2}))